## Setup

In [4]:
FINE_TUNE_ID = "ft-IMjooJPCPFHKUSONoaLTLZvh"
OPENAI_API_KEY_FILE = "MyDrive/colab/XAI/OPENAI_API_KEY_pa.txt"
DRIVE_MOUNT_PATH = '/drive/'
PROCESSED_DATA_FOLDER = "MyDrive/public/TUM XAI/Classifiers/GPT3_classifier/data/"

from google.colab import drive
drive.mount(DRIVE_MOUNT_PATH)

Drive already mounted at /drive/; to attempt to forcibly remount, call drive.mount("/drive/", force_remount=True).


In [5]:
!pip install --upgrade openai transformers shap lime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached openai-0.26.4.tar.gz (55 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 78.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.9/575.9 KB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.7/275.7 KB 32.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 95.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 27.8 MB/s eta 0:00:00
  Created wheel for openai: filename=openai-0.26.4-py3-none-any.whl size=67744 sha256=b1b3c3eec8f89231fe1b5f687ae52c147c47d30beb30ac3b9fe612a234d23a88
  Stored in directory: /root/.cache/pip/wheels/2b/d8/4e/268f029bd3277c1dd9e8781a0e0296

In [6]:
import os
import random
import sqlite3
import requests
import time
from shutil import copytree, copyfile
from pathlib import Path
from typing import Optional
from datetime import datetime
from io import BytesIO

import numpy as np
import pandas as pd
import tensorflow as tf
import openai
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import GPT2TokenizerFast

tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")

SEED: int = 42
random.seed(SEED)
os.environ['PYTHONHASHSEED'] = str(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)
tf.compat.v1.set_random_seed(SEED)

In [7]:
if os.path.exists('OPENAI_API_KEY.txt'): os.remove('OPENAI_API_KEY.txt')
copyfile(DRIVE_MOUNT_PATH + OPENAI_API_KEY_FILE, 'OPENAI_API_KEY.txt')
openai.api_key_path = 'OPENAI_API_KEY.txt'
openai_api_key = None
with open('OPENAI_API_KEY.txt', 'r') as f:
  openai_api_key = f.read()

In [8]:
if not os.path.exists('data'):
  os.mkdir('data')
  
copyfile(DRIVE_MOUNT_PATH + PROCESSED_DATA_FOLDER + "test.csv", 'data/test.csv')
df_test = pd.read_csv('data/test.csv')
df_test["source"].value_counts()

real         10763
chatgpt       3410
scigen        3366
gpt2          3100
galactica     2083
Name: source, dtype: int64

In [9]:
def prepare_data(df: pd.DataFrame) -> pd.DataFrame:
  # Add completion
  if "completion" not in df.columns:
    df["completion"] = " 1"
    df.loc[df["source"] == "real", "completion"] = " 0"

  # Create prompt
  if "prompt" not in df.columns:
    df["prompt"] = "Title:\n" + df["title"] \
      + "\n\nAbstract:\n" + df["abstract"] \
      + "\n\nIntroduction:\n" + df["introduction"] \
      + "\n\nConclusion:\n" + df["conclusion"] \
      + "\n\n###\n\n"

  return df[["prompt", "completion"]]

df_test_prepared = prepare_data(df_test)
df_test_prepared["completion"].value_counts()

 1    11959
 0    10763
Name: completion, dtype: int64

In [10]:
response = openai.FineTune.retrieve(id=FINE_TUNE_ID)
model_id = response["fine_tuned_model"]
model_id

'ada:ft-personal-2023-01-25-14-14-44'

## Testing

In [ ]:
TEST_SIZE = 1000

random_indicies = np.random.choice(df_test_prepared.index.values, TEST_SIZE)
df_test_selected = df_test_prepared.iloc[random_indicies]
df_test_selected

In [ ]:
df_test_selected['tokens'] = df_test_selected.apply(lambda row: len(tokenizer(row['prompt'])['input_ids']), axis=1)
print("The following are too long:")
df_test_selected.loc[df_test_selected['tokens'] > 2048]

In [ ]:
df_test_selected = df_test_selected.drop(df_test_selected[df_test_selected['tokens'] > 2048].index)
df_test_selected

In [ ]:
response = openai.Completion.create(model=model_id, prompt=df_test_selected['prompt'].tolist(), max_tokens=1, temperature=0, logprobs=None)

In [ ]:
results = pd.DataFrame(response["choices"])
assert len(results) == len(df_test_selected) 
results = results.set_index('index')
results = results.drop(labels=['logprobs', 'finish_reason'], axis=1)
results = results.rename({'text': 'predicted'}, axis=1)
results['correct'] = df_test_selected["completion"].values
results['prompt'] = df_test_selected["prompt"].values
results

In [ ]:
results.loc[results['predicted'] != results['correct']]

## Explainability